# Zad. 

Otagować (czyli wskazać słowa "kluczowe", Latent Dirichlet Allocation) teksty korpusu 20newsgroups.

In [1]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

### Można sczytać dane bez nagłówków i stopek

In [2]:
to_remove= ('headers', 'footers', 'quotes')

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='all', shuffle=True, remove = to_remove)

In [3]:
newsgroups_train.data[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [4]:
print(newsgroups_train.data[0])



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [5]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
print(len(newsgroups_train.target_names))

20


In [7]:
print(newsgroups_train.target)

[10  3 17 ...  3  1  7]


# Zad
Wykonaj przekształcenie Tf-Idf

```python
from sklearn.feature_extraction.text import TfidfVectorizer
```

In [19]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def remove_punctuation(words):
    remove_list = stopwords.words('english') + list(string.punctuation) + ["--", "\"", "``" , "...", "''", "'" ]
    return [w for w in words if not w in set(remove_list)]

def remove_nummbers(words):
    return [w for w in words if not w.isdigit()]

def to_lower(words):
    return [w.lower() for w in words]


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = remove_punctuation(tokens)
    tokens = remove_nummbers(tokens)
    tokens = to_lower(tokens)
    stemmer = PorterStemmer()
    res = [stemmer.stem(word) for word in tokens]
    return res 


tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
tfidf_matrix = tfidf_vectorizer.fit_transform(newsgroups_train.data)
# print( tfidf_vectorizer.vocabulary_ )

In [20]:
print("----")
print(tfidf_matrix.todense().shape)
print(tfidf_matrix.todense())

----
(18846, 175343)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Zad 
Stwórz model języka w bibliotece **gensim** na podstawie reprezentacji **Tf-Idf**

In [21]:
corpus_sparse = matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)
dictionary_list = tfidf_vectorizer.get_feature_names()
dictionary_sparse_dict = {i:j for i,j in enumerate(dictionary_list)}
dictionary = corpora.Dictionary.from_corpus(corpus_sparse, id2word=dictionary_sparse_dict)

# Zad 
Stwórz model **Lda** (Latent Dirichlet Allocation) z
* **num_topics=20**

In [22]:
model = models.LdaModel(corpus=corpus_sparse, id2word=dictionary, num_topics=20)#, alpha='auto', eta='auto', iterations=100)

In [23]:
print(len(model.get_topics()))
model.get_topics()

20


array([[5.0290391e-06, 5.0290391e-06, 5.0290391e-06, ..., 5.0290391e-06,
        5.0290391e-06, 5.0290391e-06],
       [2.1675492e-06, 2.1675485e-06, 2.1675485e-06, ..., 2.1675485e-06,
        2.1675517e-06, 2.1675485e-06],
       [5.3761805e-06, 5.3761833e-06, 5.3761805e-06, ..., 5.3761805e-06,
        5.3761805e-06, 5.3761805e-06],
       ...,
       [5.6447743e-06, 5.6447743e-06, 5.6447743e-06, ..., 5.6447743e-06,
        5.6447743e-06, 5.6447743e-06],
       [5.6230783e-06, 5.6230783e-06, 5.6230783e-06, ..., 5.6230783e-06,
        5.6230783e-06, 5.6230783e-06],
       [5.4819689e-06, 5.4819689e-06, 5.4819689e-06, ..., 5.4819689e-06,
        5.4819689e-06, 5.4819689e-06]], dtype=float32)

# Zad

Wypisz z czego składają się tematy:

In [24]:
import numpy as np

topic_inex = 1

model.print_topic(topicno=topic_inex)

'0.006*"god" + 0.004*"christian" + 0.003*"\'s" + 0.003*"n\'t" + 0.002*"jew" + 0.002*"church" + 0.002*"jesu" + 0.002*"believ" + 0.002*"peopl" + 0.002*"bibl"'

In [25]:
topic2_terms = model.get_topic_terms(topicid=topic_inex)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

[('god', 0.0061011775),
 ('christian', 0.0035652139),
 ("'s", 0.0029952528),
 ("n't", 0.002932407),
 ('jew', 0.0021863314),
 ('church', 0.0021611564),
 ('jesu', 0.0021005387),
 ('believ', 0.0020734333),
 ('peopl', 0.002051582),
 ('bibl', 0.002028195)]

# Zad 

Proszę posortować zdania najbardziej pasujące do danego tematu. 

In [27]:
import gensim


corpus_lda = model[corpus_sparse] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
# for d in corpus_lda:
#     print(d)
numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=20)

In [28]:
top_inex = 4

print(model.print_topic(topicno=top_inex))

docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
print("----------------------")
for x in docs[:3]:
    print(newsgroups_train.data[x])
    print("----------------------")

0.002*"bmw" + 0.001*"ray" + 0.001*"ir" + 0.001*"repair" + 0.001*"h+" + 0.001*"pat" + 0.001*"bo" + 0.001*"babe" + 0.001*"pcx" + 0.001*"bp"
----------------------
From article <qekN3B4w165w@sys6626.bison.mb.ca>, by baden@sys6626.bison.mb.ca (baden de bari):
----------------------
Playoff leaders as of April 19, 1993

    Player       Team   GP  G   A  Pts +/- PIM

    M.Lemieux    PIT     1   2   2   4   0   0
    Juneau       BOS     1   1   3   4   0   0
    Noonan       CHI     1   3   0   3   0   0
    Mogilny      BUF     1   2   1   3   0   0
    Neely        BOS     1   2   1   3   0   0
    Brown        STL     1   1   2   3   0   0
    Jagr         PIT     1   1   2   3   0   0
    Oates        BOS     1   0   3   3   0   0
    Carson       LA      1   2   0   2   0   0
    Hunter       WAS     1   2   0   2   0   0
    Stevens      NJ      1   2   0   2   0   0
    Cullen       TOR     1   1   1   2   0   0
    Hull         STL     1   1   1   2   0   0
    Khristich    WAS    

# Wizualizacja modelu LDA:

pyLDAvis

In [29]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [30]:
pyLDAvis.gensim.prepare(model, corpus_sparse, dictionary)

C:\Users\admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.399542 -0.050237       1        1  61.508072
1     -0.196914  0.107687       2        1  13.242567
14    -0.021701 -0.053935       3        1   4.781486
0      0.026614 -0.004881       4        1   2.034768
8      0.034246 -0.000130       5        1   1.399019
2      0.032860 -0.001411       6        1   1.383515
10     0.033538 -0.002872       7        1   1.325752
7      0.039361  0.000055       8        1   1.230309
19     0.033627 -0.000293       9        1   1.216158
4      0.035824 -0.000184      10        1   1.202910
12     0.034625 -0.001047      11        1   1.201344
9      0.035273 -0.000792      12        1   1.129436
15     0.036681 -0.000631      13        1   1.123096
18     0.039141  0.001280      14        1   1.064899
5      0.038881  0.000610      15        1   1.043213
3      0.037652  0.001526      16        1   1.035114
11     0.038776  0.001194      17        1   1.024291
16     0.040107  0.001239      18        1   1.024106
17     0.040936  0.001357      19        1   1.015008
6      0.040015  0.001464      20        1   1.014945, topic_info=       Category        Freq  \
term                          
89012   Default  103.000000   
68354   Default   57.000000   
3165    Default  420.000000   
121200  Default  439.000000   
61900   Default  104.000000   
148640  Default   37.000000   
68445   Default   31.000000   
100455  Default   33.000000   
100433  Default   31.000000   
131103  Default  208.000000   
143990  Default   29.000000   
144399  Default  166.000000   
62467   Default   29.000000   
160574  Default  111.000000   
93870   Default   35.000000   
61897   Default   30.000000   
147255  Default   25.000000   
127256  Default  308.000000   
140229  Default   39.000000   
118255  Default   26.000000   
106003  Default   67.000000   
108016  Default   45.000000   
77617   Default   16.000000   
167701  Default  344.000000   
154976  Default  208.000000   
167553  Default   67.000000   
106947  Default   46.000000   
58129   Default   22.000000   
107361  Default   63.000000   
98932   Default   39.000000   
...         ...         ...   
140713  Topic20    0.858053   
87404   Topic20    0.657816   
56756   Topic20    0.528704   
61540   Topic20    0.500200   
87830   Topic20    0.483475   
56060   Topic20    0.383770   
70293   Topic20    0.346367   
146167  Topic20    0.209754   
148537  Topic20    0.200687   
65825   Topic20    0.165342   
123493  Topic20    0.161406   
122639  Topic20    0.147549   
71118   Topic20    0.126402   
83422   Topic20    0.111476   
154002  Topic20    0.075301   
159200  Topic20    0.282232   
58384   Topic20    0.053715   
114597  Topic20    0.041368   
3796    Topic20    0.040217   
154669  Topic20    0.036168   
45785   Topic20    0.035024   
151947  Topic20    0.034915   
103719  Topic20    0.030881   
171879  Topic20    0.032338   
115284  Topic20    0.029202   
94178   Topic20    0.033304   
61738   Topic20    0.028118   
155230  Topic20    0.060418   
71594   Topic20    0.044234   
61107   Topic20    0.044273   

                                                     Term       Total  \
term                                                                    
89012                                                 god  103.000000   
68354                                           christian   57.000000   
3165                                                   's  420.000000   
121200                                                n't  439.000000   
61900                                              believ  104.000000   
148640                                               soon   37.000000   
68445                                              church   31.000000   
100455                                                jew   33.000000   
100433                                               jesu   31.000000   
131103      